#Marcin Staśko Submission for Capstone Project - Eductation profiling

In [ ]:
import riiideducation
# import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
try:
    env = riiideducation.make_env()
except:
    pass
import seaborn as sns
sns.set_style("dark")

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[1, 2, 3, 4, 7, 8, 9],
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [ ]:
#removing True or 1 for content_type_id

train = train[train.content_type_id == False]

#arrange by timestamp

train = train.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

train.head(10)

In [ ]:
train['timestamp'].hist(bins = 100)

In [ ]:
# Answers by by user

train_questions_only_df = train[train['answered_correctly']!=-1]
train_questions_only_df['answered_correctly'].mean()
grouped_by_user_df = train_questions_only_df.groupby('user_id')
user_answers_df = grouped_by_user_df.agg({'answered_correctly': ['mean', 'count'] })

user_answers_df[('answered_correctly','mean')].hist(bins = 100)

In [ ]:
#getting final results ready for later, so we can clear memory
results_c_final = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c_final.columns = ["answered_correctly_content"]

results_u_final = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count'])
results_u_final.columns = ['answered_correctly_user', 'sum', 'count']

In [ ]:
#saving value to fillna
elapsed_mean = train.prior_question_elapsed_time.mean()
train.drop(['timestamp', 'content_type_id'], axis=1, inplace=True)

In [ ]:
validation = pd.DataFrame()

In [ ]:
for i in range(4):
    last_records = train.drop_duplicates('user_id', keep = 'last')
    train = train[~train.index.isin(last_records.index)]
    validation = validation.append(last_records)

In [ ]:
X = pd.DataFrame()
for i in range(15):
    last_records = train.drop_duplicates('user_id', keep = 'last')
    train = train[~train.index.isin(last_records.index)]
    X = X.append(last_records)

In [ ]:
results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count'])
results_u.columns = ["answered_correctly_user", 'sum', 'count']

In [ ]:
#clearing memory
del(train)

In [ ]:
X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c_final, on=['content_id'], how="left")
validation = pd.merge(validation, results_u, on=['user_id'], how="left")
validation = pd.merge(validation, results_c_final, on=['content_id'], how="left")

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

X.prior_question_had_explanation.fillna(False, inplace = True)
validation.prior_question_had_explanation.fillna(False, inplace = True)

validation["prior_question_had_explanation_enc"] = lb_make.fit_transform(validation["prior_question_had_explanation"])
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])
X.head()

In [ ]:
#reading in question df
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                            usecols=[0, 3],
                            dtype={'question_id': 'int16',
                              'part': 'int8'}
                          )

In [ ]:
X = pd.merge(X, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
validation = pd.merge(validation, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
X.part = X.part - 1
validation.part = validation.part - 1

In [ ]:
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)
X.head()

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [ ]:
X = X[['answered_correctly_user', 'answered_correctly_content', 'sum', 'count',
       'prior_question_elapsed_time','prior_question_had_explanation_enc', 'part']]
X_val = X_val[['answered_correctly_user', 'answered_correctly_content', 'sum', 'count',
       'prior_question_elapsed_time','prior_question_had_explanation_enc', 'part']]

In [ ]:
# Filling with 0.5 for simplicity; there could likely be a better value
X['answered_correctly_user'].fillna(0.5,  inplace=True)
X['answered_correctly_content'].fillna(0.5,  inplace=True)

X['part'].fillna(4, inplace = True)
X['sum'].fillna(0, inplace = True)
X['count'].fillna(0, inplace = True)
X['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
X['prior_question_had_explanation_enc'].fillna(0, inplace = True)

In [ ]:
# Filling with 0.5 for simplicity; there could likely be a better value
X_val['answered_correctly_user'].fillna(0.5,  inplace=True)
X_val['answered_correctly_content'].fillna(0.5,  inplace=True)

X_val['part'].fillna(4, inplace = True)
X_val['count'].fillna(0, inplace = True)
X_val['sum'].fillna(0, inplace = True)
X_val['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
X_val['prior_question_had_explanation_enc'].fillna(0, inplace = True)

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'max_bin': 255,
    'learning_rate': 0.09,
    'num_leaves': 6,
    'boosting_type':'gbdt', 
    'colsample_bytree' : 0.64,
    'subsample' : 0.7,
    'reg_alpha' : 1,
    'reg_lambda' : 1,
    'min_child_weight': 1, 
    'min_child_samples': 5,
    'min_split_gain': 0.5,
    'metric' : 'binary_error',
    'lambda_l1': 5, # L1 regularization
    'lambda_l2': 10, # L2 regularization
    }

lgb_train = lgb.Dataset(X, y, categorical_feature = ['part', 'prior_question_had_explanation_enc'])
lgb_eval = lgb.Dataset(X_val, y_val, categorical_feature = ['part', 'prior_question_had_explanation_enc'], reference=lgb_train)

In [ ]:
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=50,
    num_boost_round=10000,
    early_stopping_rounds=12
)

In [ ]:
y_pred = model.predict(X_val)
y_true = np.array(y_val)
roc_auc_score(y_true, y_pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#displaying the most important features
lgb.plot_importance(model)
plt.show()

In [ ]:
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df = pd.merge(test_df, results_u_final, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_c_final, on=['content_id'],  how="left")
    test_df['answered_correctly_user'].fillna(0.5,  inplace=True)
    test_df['answered_correctly_content'].fillna(0.5,  inplace=True)
    test_df['part'] = test_df.part - 1

    test_df['part'].fillna(4, inplace = True)
    test_df['sum'].fillna(0, inplace=True)
    test_df['count'].fillna(0, inplace=True)
    test_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
    test_df['answered_correctly'] =  model.predict(test_df[['answered_correctly_user', 'answered_correctly_content', 'sum', 'count',
                                                                  'prior_question_elapsed_time','prior_question_had_explanation_enc', 'part']])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])